In [1]:
%pylab inline
import keras
from sklearn import utils
from hyperopt import hp, fmin, tpe

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [2]:
xy = np.load('data/xy.npz')

x = xy["x"]
y = xy["y"]

num_cat = y.max() + 1

classes = np.arange(num_cat)
cw = utils.class_weight.compute_class_weight("balanced",classes.flat, y.flat)
cw = dict(zip(classes,cw))

In [2]:
xy = np.load('data/xy-spec.npz')

x = xy["x"]
y = xy["y"]

num_cat = y.max() + 1

classes = np.arange(num_cat)
cw = utils.class_weight.compute_class_weight("balanced",classes.flat, y.flat)
cw = dict(zip(classes,cw))

In [3]:
space = {
    "dropout_prob": hp.uniform("dropout_prob", 0,1),
    "batch_size": hp.choice("batch_size", )
}

In [3]:
dropout_prob = .5
init_stddev = 0.01

kernel_size = (4,4)

cnn_layers =[
    keras.layers.InputLayer(input_shape=x.shape[1:]),

    # Conv layer
    keras.layers.Conv2D(
        64,
        kernel_size=kernel_size,
        padding="same",
        activation="elu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout_prob),
    keras.layers.MaxPool2D(pool_size=(2, 2), padding="same"),
    
    keras.layers.Conv2D(
        64,
        kernel_size=kernel_size,
        padding="same",
        activation="elu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout_prob),
    keras.layers.MaxPool2D(pool_size=(2, 2), padding="same"),
    
    keras.layers.Conv2D(
        32,
        kernel_size=kernel_size,
        padding="same",
        activation="elu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout_prob),
    keras.layers.MaxPool2D(pool_size=(2, 2), padding="same"),
    
    keras.layers.Conv2D(
        16,
        kernel_size=kernel_size,
        padding="same",
        activation="elu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout_prob),
    keras.layers.MaxPool2D(pool_size=(2, 2), padding="same"),
    
    keras.layers.Flatten(),

    # Hidden Layer 1
    keras.layers.Dense(
        128,
        activation="elu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.Dropout(dropout_prob),

    # Classification Layer
    keras.layers.Dense(
        num_cat,
        activation="softmax",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
]
cnn_model = keras.Sequential(cnn_layers)
cnn_model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 129, 124, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 129, 124, 64)      1088      
_________________________________________________________________
batch_normalization_1 (Batch (None, 129, 124, 64)      256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 129, 124, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 65, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 65, 62, 64)        65600     
_________________________________________________________________
batch_normalization_2 (Batch (None, 65, 62, 64)        256       
__________

In [5]:
batch_size = 256

callbacks = [
    keras.callbacks.EarlyStopping(patience=3, verbose=1),
    keras.callbacks.ReduceLROnPlateau(
        factor=0.2, patience=1, verbose=1, min_lr=1e-6)
]

cnn_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='nadam',
    metrics=['accuracy'])

history = cnn_model.fit(
    batch_size=batch_size,
    x=x,
    y=y,
    class_weight=cw,
    validation_data=(xy["x_val"], xy["y_val"]),
    epochs=100,
    verbose=1,
    callbacks=callbacks,
    initial_epoch=0)
np.min(history.history["val_loss"])

Train on 52888 samples, validate on 7058 samples
Epoch 1/100
52888/52888 [==============================] - 286s 5ms/step - loss: 2.2057 - acc: 0.0793 - val_loss: 15.5243 - val_acc: 0.0368
Epoch 3/100
52736/52888 [============================>.] - ETA: 0s - loss: 2.2034 - acc: 0.0828
Epoch 00003: reducing learning rate to 0.0004000000189989805.
52888/52888 [==============================] - 286s 5ms/step - loss: 2.2035 - acc: 0.0836 - val_loss: 15.5243 - val_acc: 0.0368
Epoch 4/100
52888/52888 [==============================] - 286s 5ms/step - loss: 2.2173 - acc: 0.0896 - val_loss: 2.3534 - val_acc: 0.0731
Epoch 5/100
52888/52888 [==============================] - 286s 5ms/step - loss: 2.1983 - acc: 0.0912 - val_loss: 6.6651 - val_acc: 0.0368
Epoch 6/100
52736/52888 [============================>.] - ETA: 0s - loss: 2.1884 - acc: 0.0848
Epoch 00006: reducing learning rate to 8.000000379979611e-05.
52888/52888 [==============================] - 286s 5ms/step - loss: 2.1893 - acc: 0.0848

2.3534297524919854

In [33]:
dropout_prob = .5
init_stddev = 0.01

cnn_layers =[
    keras.layers.InputLayer(input_shape=x.shape[1:]),
    
    keras.layers.BatchNormalization(),

    # Conv layer
    keras.layers.Conv2D(
        186,
        kernel_size=(8, x.shape[2]),
        padding="valid",
        activation="elu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout_prob),
    
    keras.layers.Conv2D(
        186,
        kernel_size=(8, 1),
        strides=(4, 1),
        padding="valid",
        activation="elu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout_prob),
    
    keras.layers.Conv2D(
        186,
        kernel_size=(4, 1),
        strides=(1, 1),
        padding="valid",
        activation="elu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout_prob),
    
    keras.layers.Conv2D(
        186,
        kernel_size=(4, 1),
        strides=(2, 1),
        padding="valid",
        activation="elu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout_prob),
    
    keras.layers.Flatten(),

    # Hidden Layer 1
    keras.layers.Dense(
        500,
        activation="elu",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
    keras.layers.Dropout(dropout_prob),

    # Classification Layer
    keras.layers.Dense(
        num_cat,
        activation="softmax",
        kernel_initializer=keras.initializers.TruncatedNormal(
            stddev=init_stddev)),
]
cnn_model = keras.Sequential(cnn_layers)
cnn_model.summary()

callbacks = [
    keras.callbacks.EarlyStopping(patience=3, verbose=1),
    keras.callbacks.ReduceLROnPlateau(
        factor=0.2, patience=1, verbose=1, min_lr=1e-6)
]

cnn_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='nadam',
    metrics=['accuracy'])

history = cnn_model.fit(
    x=x,
    y=y,
    class_weight=cw,
    validation_data=(xy["x_val"], xy["y_val"]),
    epochs=100,
    verbose=1,
    callbacks=callbacks,
    initial_epoch=0)
np.min(history.history["val_loss"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 128, 32, 1)        0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 128, 32, 1)        4         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 121, 1, 186)       47802     
_________________________________________________________________
batch_normalization_20 (Batc (None, 121, 1, 186)       744       
_________________________________________________________________
dropout_62 (Dropout)         (None, 121, 1, 186)       0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 29, 1, 186)        276954    
_________________________________________________________________
batch_normalization_21 (Batc (None, 29, 1, 186)        744       
__________

1.5142989171783423